### 预先定义的函数

In [1]:
# 读入n, e, c
ciphers = {}
for i in range(21):
    f = open("./frame/Frame{}".format(i),'r')
    data = f.read()
    data = [int(data[i * 256 : (i + 1) * 256], 16) for i in range(3)]
    cipher = dict(zip(['n', 'e', 'c'], data))
    ciphers[i] = cipher

# 根据p, q, e, c解密
def decrypt(p, q, e, c):
    fai = (p - 1) * (q - 1)
    d = inverse(e, fai)
    return pow(c, d, p * q)
def message_print(m):
    return bytes.fromhex(hex(m)[2:])[-8:]

In [2]:
from Crypto.Util.number import inverse,isPrime,getPrime
from gmpy2 import gcdext,iroot,isqrt,gcd
from random import randint

### 公模攻击

In [43]:
can_attack = []
for i in range(len(ciphers)):
    for j in range(i, len(ciphers)):
        if ciphers[i]['n'] == ciphers[j]['n']\
            and gcd(ciphers[i]['e'], ciphers[j]['e']) == 1:
            can_attack.append((i, j))
can_attack

[(0, 4)]

In [44]:
def common_mod(e1, e2, n, c1, c2):
    g, s, t = gcdext(e1, e2)
    if s < 0:
        s = -s
        c1 = inverse(c1, n)
    if t < 0:
        t = -t
        c2 = inverse(c2, n)
    return pow(c1, s, n) * pow(c2, t, n) % n

In [45]:
m = common_mod(ciphers[0]['e'], ciphers[4]['e'], ciphers[0]['n'], ciphers[0]['c'], ciphers[4]['c'])
message_print(m)

b'My secre'

### 模不互素

In [46]:
can_attack = []
for i in range(len(ciphers)):
    for j in range(i + 1, len(ciphers)):
        if ciphers[i]['n'] !=ciphers[j]['n'] and gcd(ciphers[i]['n'], ciphers[j]['n']) != 1:
            can_attack.append((i, j))
can_attack

[(1, 18)]

In [47]:
def common_factor(c1, c2, n1, n2, e1, e2):
    p = gcd(n1, n2)
    q1 = n1 // p
    q2 = n2 // p
    m1 = decrypt(p, q1, e1, c1)
    m2 = decrypt(p, q2, e2, c2)
    return m1, m2

In [48]:
f0, f1 = ciphers[1], ciphers[18]
m0, m1 = common_factor(f0['c'], f1['c'], f0['n'], f1['n'], f0['e'], f1['e'])


In [49]:
print(message_print(m0))
print(message_print(m1))

b'. Imagin'
b'm A to B'


### 低指数加密攻击

In [51]:
e_list = {}
for i in range(len(ciphers)):
    cipher = ciphers[i]
    if cipher['e'] not in e_list:
        e_list[cipher['e']] = [i]
    else:
        e_list[cipher['e']].append(i)
for e, idx in e_list.items():
    print(f"{e} : {idx}")

46786465362686334917265996843779843233606992585424976481745055335468678697948774988450305612127967926533923268260412557000125153569622340353246096040604284883505587337829322949633637609180797447754513992039018904786537115087888005528547900640339270052628915440787357271345416818313808448127098885767015748889 : [0]
65537 : [1, 2, 5, 6, 9, 10, 13, 14, 17, 18, 19]
5 : [3, 8, 12, 16, 20]
152206992575706893484835984472544529509325440944131662631741403414037956695665533186650071476146389737020554215956181827422540843366433981607643940546405002217220286072880967331118344806315756304650248634546597784597963886656422706197757265316981889118026978865295597135470735576032282694348773714479076093197 : [4]
3 : [7, 11, 15]


In [118]:
def CRT(a, m):
    M = 1
    for i in m:
        M *= i
    M_list = [M // i for i in m]
    y_list = [inverse(M_list[i], m[i]) for i in range(len(m))]
    x = 0
    for i in range(len(m)):
        x += a[i] * M_list[i] * y_list[i]
    return x % M

c5_list = [cipher['c'] for cipher in ciphers.values() if cipher['e'] == 5]
n5_list = [cipher['n'] for cipher in ciphers.values() if cipher['e'] == 5]
m = CRT(c5_list, n5_list)
m = iroot(m, 5)[0]
message_print(m)

b't is a f'

In [58]:
c5_list = [cipher['c'] for cipher in ciphers.values() if cipher['e'] == 5]
t is a fn5_list = [cipher['n'] for cipher in ciphers.values() if cipher['e'] == 5]
m = CRT(c5_list, n5_list)
m = iroot(m, 5)[0]
message_print(m)

b't is a f'

### 费马分解

In [119]:
def fermat_factor(n, limit=100000):
    x = iroot(n, 2)[0]
    while x ** 2 < n:
        x += 1
    for i in range(x, x + limit):
        y = i * i - n
        if isqrt(y) ** 2 == y:
            return i - isqrt(y), i + isqrt(y)
    return None, None

In [120]:
for i in range(len(ciphers)):
    cipher = ciphers[i]
    p, q = fermat_factor(cipher['n'])
    if p is not None:
        print(f"Frame{i} : {p}, {q}")
        print(message_print(decrypt(p, q, cipher['e'], cipher['c'])))

Frame10 : 9686924917554805418937638872796017160525664579857640590160320300805115443578184985934338583303180178582009591634321755204008394655858254980766008932978633, 9686924917554805418937638872796017160525664579857640590160320300805115443578184985934338583303180178582009591634321755204008394655858254980766008932978699
b'will get'


### p - 1 光滑

In [3]:
def pollard_p_1(N, B = 2220000):
    a = 2
    n = 2
    for i in range(2, B):
        a = pow(a, i, N)
        d = gcd(a - 1, N)
        if d != 1 and d != N:
            return d

In [4]:
for i in range(len(ciphers)):
    cipher = ciphers[i]
    p = pollard_p_1(cipher['n'])
    if p is not None:
        q = cipher['n'] // p
        print(f'Frame{i}可以破解')
        print(message_print(decrypt(p, q, cipher['e'], cipher['c'])))

Frame2可以破解
b' That is'
Frame6可以破解
b' "Logic '
Frame19可以破解
b'instein.'
